In [ ]:
import pandas as pd
dt = pd.read_csv('./train.csv')

In [ ]:
dt.head()

In [ ]:
# Lets write a function to get every title in our Name column
def get_title(name):
    if "." in name:
        return name.split(",")[1].split(".")[0].strip()
    else:
        return "Unknown"

titles = sorted(set([x for x in dt.Name.map(lambda x: get_title(x))]) )
print(titles)


In [ ]:
def replace_titles(data):
    title = data['Title']
    if title in['Capt', 'Col', 'Major']:
        return 'Officer'
    elif title in['Don', 'Dona', 'Jonkheer', 'Lady', 'Sir', 'the Countess']:
        return 'Royalty'
    elif title in ['Mile', 'Ms', 'Mme']:
        return 'Ms'
    else:
        return title
    
# Lets create a new column that we'll add to our table    
dt['Title'] = dt.Name.map(lambda x: get_title(x))
# Apply the title column which we created above
dt['Title'] = dt.apply(replace_titles, axis=1)
print(dt.Title.value_counts())

In [ ]:
dt['Sex'].replace(('male', 'female'), (0, 1), inplace=True)
dt['Embarked'].replace(('S', 'C', 'Q'), (0, 1, 2), inplace=True)
dt['Title'].replace(('Mr', 'Miss', 'Mrs', 'Master', 'Dr', 'Rev', 'Officer', 'Royalty', 'Mme', 'Mile'), (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), inplace=True)
dt.head()

In [ ]:
dt['Age'].fillna(dt['Age'].median(), inplace=True)
dt['Fare'].fillna(dt['Fare'].median(), inplace=True)
dt['Embarked'].fillna('S', inplace=True)
dt.tail()

In [ ]:
dt.drop('Name', axis=1, inplace=True)
dt.drop('Cabin', axis=1, inplace=True)
dt.drop('Ticket', axis=1, inplace=True)
dt.head()

In [ ]:
print(dt.isnull().sum())
print(dt['Sex'].sample(5))
print(dt['Embarked'].sample(5))

In [ ]:
dt.info()

In [89]:
# y is the dependent variable. Y the dependent variable is always the variable
# we're trying to predict
y = dt["Survived"]
# x is the independent variables. X is a combination of all columns except Survived and PassengerId
# so we drop and remove both columns
x = dt.drop(["Survived", "PassengerId"], axis=1)


In [94]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x,y,test_size=0.1)

In [91]:
# Get names of indexes for which column Title has value Mlle
indexNames = dt[dt['Title'] == 'Mlle'].index
# Delete these row indexes from dataFrame
dt.drop(indexNames , inplace=True)

KeyError: 'Title'

In [ ]:
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

randomforest = RandomForestClassifier()
randomforest.fit(x_train, y_train)
y_pred = randomforest.predict(x_val)
acc_randomforest = round(accuracy_score(y_pred, y_val) * 100, 2)
print("Accuracy: {}".format(acc_randomforest))


In [98]:
dt

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,0,22.0,1,0,7.2500,0
1,2,1,1,1,38.0,1,0,71.2833,1
2,3,1,3,1,26.0,0,0,7.9250,0
3,4,1,1,1,35.0,1,0,53.1000,0
4,5,0,3,0,35.0,0,0,8.0500,0
...,...,...,...,...,...,...,...,...,...
886,887,0,2,0,27.0,0,0,13.0000,0
887,888,1,1,1,19.0,0,0,30.0000,0
888,889,0,3,1,28.0,1,2,23.4500,0
889,890,1,1,0,26.0,0,0,30.0000,1


In [106]:
dt_test = pd.read_csv('test.csv')

ids = dt_test['PassengerId']

dt_test['Age'].fillna(dt['Age'].median(), inplace=True)
dt_test['Fare'].fillna(dt['Fare'].median(), inplace=True)
dt_test['Embarked'].fillna('S', inplace=True)
dt_test.drop('Name', axis=1, inplace=True)
dt_test.drop('Cabin', axis=1, inplace=True)
dt_test.drop('Ticket', axis=1, inplace=True)
dt_test.drop('PassengerId', axis=1, inplace=True)
dt_test['Sex'].replace(('male', 'female'), (0, 1), inplace=True)
dt_test['Embarked'].replace(('S', 'C', 'Q'), (0, 1, 2), inplace=True)
dt_test.head()


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,34.5,0,0,7.8292,2
1,3,1,47.0,1,0,7.0000,0
2,2,0,62.0,0,0,9.6875,2
3,3,0,27.0,0,0,8.6625,0
4,3,1,22.0,1,1,12.2875,0


In [107]:
predictions = randomforest.predict(dt_test)
output = pd.DataFrame({'PassengerId': ids, 'Survived' : predictions})
output.to_csv('submissions.csv', index=False)